In [1]:
import requests
import json
import os
import numpy as np
from openai import OpenAI
from typing import List, Dict
import time

# Première API pour récupérer tous les ids récents

In [2]:
def search_offers(limit=100, skip=0):
    # URL de l'API
    url = "https://civiweb-api-prd.azurewebsites.net/api/Offers/search"

    # Corps de la requête (payload)
    payload = {
        "limit": limit,
        "skip": skip,
        "latest": ["true"],
        "method": ["null"],
        "activitySectorId": [],
        "missionsTypesIds": [],
        "missionsDurations": [],
        "gerographicZones": [],
        "countriesIds": [],
        "studiesLevelId": [],
        "companiesSizes": [],
        "specializationsIds": [],
        "entreprisesIds": [0],
        "missionStartDate": None,
        "query": None
    }

    # Headers
    headers = {
        "Content-Type": "application/json"
    }

    # Effectuer la requête POST
    response = requests.post(url, json=payload, headers=headers)
    # Vérifier le code de statut
    response.raise_for_status()
    # Afficher le résultat
    print(f"Code de statut: {response.status_code}")
        
    return response.json()

#search_offers(2,0)
    

# Appelle par Id

In [19]:
def get_offer_details(offer_id):
    url = f"https://civiweb-api-prd.azurewebsites.net/api/Offers/details/{offer_id}"
    # Headers (optionnel pour un GET simple)
    headers = {
        "Accept": "application/json"
    }
    # Effectuer la requête GET
    response = requests.get(url, headers=headers)
    # Vérifier le code de statut
    response.raise_for_status()
    print("   Récupération des données réussies")
    return response.json()

get_offer_details(230682)

    

   Récupération des données réussies


{'id': 230682,
 'organizationName': 'DEMGY NORMANDIE',
 'missionTitle': 'ERP SYSTEMS ANALYST (H/F)',
 'missionDuration': 6,
 'viewCounter': 152,
 'candidateCounter': 8,
 'missionType': 'VIE',
 'missionTypeEn': 'VIE',
 'organizationPresentation': '',
 'organizationUrlImage': '',
 'organizationImage': None,
 'organizationPathImage': None,
 'pathImage': '',
 'image': None,
 'activitySectorN1': 'INDUSTRIES CHIMIQUES ET PLASTURGIE',
 'activitySectorN2': None,
 'activitySectorN3': None,
 'activitySectorN1Id': 100008,
 'ca': '19027',
 'effectif': 159,
 'organizationCountryCounter': '',
 'organizationExpertise': None,
 'cityAffectationId': -1,
 'cityName': 'TACOMA (WA)',
 'cityNameEn': 'TACOMA (WA)',
 'activitySectorOfferId': 8,
 'levelStudyIds': None,
 'specializations': None,
 'missionDescription': 'Présentation de la société :\n\nDEMGY Pacific est une entreprise américaine spécialisée dans la fabrication de pièces plastiques et métalliques pour l’aéronautique. Basée à Tacoma (Washington), e

## Fonction pour clean les données manquantes (appelés directment dans la création des chunks)

In [4]:
def clean_offer_data(offer_data):
    print("   Nettoyage des données")
    """Nettoie et complète les données manquantes"""
    return {
        "reference": offer_data.get("reference", "N/A"),
        "organizationName": offer_data.get("organizationName", "Entreprise non spécifiée"),
        "missionTitle": offer_data.get("missionTitle", "Titre non spécifié"),
        "missionDescription": offer_data.get("missionDescription", "Description non disponible"),
        "missionProfile": offer_data.get("missionProfile", "Profil non spécifié"),
        "countryName": offer_data.get("countryName", "Pays non spécifié"),
        "cityName": offer_data.get("cityName", "Ville non spécifiée"),
        "activitySectorN1": offer_data.get("activitySectorN1", "Secteur non spécifié"),
        "missionDuration": offer_data.get("missionDuration", "Durée non spécifiée"),
        "indemnite": offer_data.get("indemnite", "Non spécifié"),
        "missionStartDate": offer_data.get("missionStartDate", "Date non spécifiée"),
        "contactEmail": offer_data.get("contactEmail", "Email non disponible"),
        "missionType": offer_data.get("missionType", "Type non spécifié")
    }

# Création des chunks pour notre RAG

In [5]:

def create_chunks_for_rag(offer_data):
    """Crée 2 chunks optimisés pour le RAG"""
    chunks = []

    # 1. Nettoyer les données brutes
    cleaned_offer = clean_offer_data(offer_data)

    # 2. Extraire les métadonnées depuis les données nettoyées
    common_metadata = {
        "offer_id": cleaned_offer.get("reference"),  
        "company": cleaned_offer.get("organizationName"),
        "title": cleaned_offer.get("missionTitle"),
        "country": cleaned_offer.get("countryName"),
        "city": cleaned_offer.get("cityName"),
        "sector": cleaned_offer.get("activitySectorN1"),
        "duration_months": cleaned_offer.get("missionDuration"),
        "salary_eur": cleaned_offer.get("indemnite"),
        "start_date": cleaned_offer.get("missionStartDate"),
        "contact_email": cleaned_offer.get("contactEmail"),
        "mission_type": cleaned_offer.get("missionType")
    }

    
    
    # CHUNK 1: Description de la mission
    chunk1_content = f"""Offre VIE {offer_data.get('reference')} - {offer_data.get('missionTitle')}
    Entreprise: {offer_data.get('organizationName')}
    Localisation: {offer_data.get('cityName')}, {offer_data.get('countryName')}
    Secteur d'activité: {offer_data.get('activitySectorN1')}
    Durée: {offer_data.get('missionDuration')} mois
    Indemnité: {offer_data.get('indemnite')} € par mois
    Description de la mission: {offer_data.get('missionDescription', 'Non spécifiée')}"""

    new_chunk = {
        "content": chunk1_content,
        "metadata": {
            **common_metadata,
            "chunk_type": "mission_description",
            "chunk_id": f"{offer_data.get('id')}_desc"
        }
    }

    print(f"   Taille du chunk de Description (caractères) : {len(new_chunk['content'])}")
    chunks.append(new_chunk)
    
    # CHUNK 2: Profil recherché
    chunk2_content = f"""Offre VIE {offer_data.get('reference')} - {offer_data.get('missionTitle')}
    Entreprise: {offer_data.get('organizationName')}
    Localisation: {offer_data.get('cityName')}, {offer_data.get('countryName')}
    Secteur: {offer_data.get('activitySectorN1')}
    Profil recherché: {offer_data.get('missionProfile', 'Non spécifié')}"""

    new_chunk2 = {
        "content": chunk2_content,
        "metadata": {
            **common_metadata,
            "chunk_type": "candidate_profile",
            "chunk_id": f"{offer_data.get('id')}_profile"
        }
    }

    chunks.append(new_chunk2)
    print(f"   Taille du chunk de Profil (caractères) : {len(new_chunk2['content'])}")
    
    return chunks

# Create the embeding of the data 

In [6]:
# Configuration OpenAI
api_key = os.environ.get("API_KEY_OPENAI")
client = OpenAI(api_key=api_key)

In [7]:
def get_text_embedding(text: str) -> List[float]:
    """
    Crée un embedding pour un texte avec OpenAI
    
    Args:
        text: Le texte à embedder
        
    Returns:
        Liste de floats représentant l'embedding
    """
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"  # 1536 dimensions, $0.02/1M tokens
    )
    return response.data[0].embedding

In [8]:
def create_embeddings_batch(chunks: List[Dict], batch_size: int = 100) -> List[Dict]:
    """
    Crée les embeddings pour tous les chunks avec traitement par batch
    
    Args:
        chunks: Liste des chunks à embedder
        batch_size: Nombre de chunks à traiter par batch (max 100 pour OpenAI)
        
    Returns:
        Liste des chunks enrichis avec leurs embeddings
    """
    enriched_chunks = []
    
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        print(f"Traitement du batch {i//batch_size + 1} ({len(batch)} chunks)...")
        
        # Extraire les contenus textuels
        texts = [chunk["content"] for chunk in batch]
        
        # Créer les embeddings en batch (plus efficace)
        response = client.embeddings.create(
            input=texts,
            model="text-embedding-3-small"
        )
        
        # Associer chaque embedding à son chunk
        for j, chunk in enumerate(batch):
            enriched_chunk = chunk.copy()
            enriched_chunk["embedding"] = response.data[j].embedding
            enriched_chunks.append(enriched_chunk)
        
        # Rate limiting (optionnel mais recommandé)
        time.sleep(0.1)
    
    print(f"✅ {len(enriched_chunks)} embeddings créés")
    return enriched_chunks




# Let's save our chunks and embeddings

In [9]:
def save_to_json(data, filename):
    """Sauvegarde les données en JSON"""
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"✅ Données sauvegardées dans {filename}")

In [10]:
def save_embeddings_numpy(chunks_with_embeddings, filename="vie_embeddings.npz"):
    """
    Sauvegarde les embeddings en format NumPy pour un chargement rapide
    """
    embeddings = np.array([chunk["embedding"] for chunk in chunks_with_embeddings])
    metadata = [chunk["metadata"] for chunk in chunks_with_embeddings]
    contents = [chunk["content"] for chunk in chunks_with_embeddings]
    
    np.savez_compressed(
        filename,
        embeddings=embeddings,
        metadata=metadata,
        contents=contents
    )
    print(f"✅ Embeddings sauvegardés dans {filename}")
    print(f"   Shape: {embeddings.shape}")


# Fonction pour charger les embeddings sauvegardés
def load_embeddings(filename="vie_embeddings.npz"):
    """Charge les embeddings depuis un fichier NumPy"""
    data = np.load(filename, allow_pickle=True)
    return {
        'embeddings': data['embeddings'],
        'metadata': data['metadata'],
        'contents': data['contents']
    }

# Let's call everything

In [11]:
print("="*80)
print("PIPELINE RAG - OFFRES VIE")
print("="*80)

# ÉTAPE 1: Rechercher les offres
print("\n[1/5] Recherche des offres VIE...")
offers_response = search_offers(limit=10)  # Ajustez la limite selon vos besoins

# La structure de retour peut varier, adaptez selon l'API
# Supposons que l'API retourne une liste d'IDs ou d'objets simplifiés
offer_ids = []
offer_ids = [item.get("id") for item in offers_response["result"] if item.get("id") is not None]

print(f"✅ {len(offer_ids)} offres trouvées")

# ÉTAPE 2: Récupérer les détails et créer les chunks
print("\n[2/5] Récupération des détails et création des chunks...")
all_chunks = []
    

for i, offer_id in enumerate(offer_ids[:10], 1):  # Limitez pour le test
    try:
        print(f"  Traitement offre {i}/{min(10, len(offer_ids))}: {offer_id}")
        offer_details = get_offer_details(offer_id)
        chunks = create_chunks_for_rag(offer_details)
        all_chunks.extend(chunks)
        time.sleep(0.2)  # Rate limiting pour l'API Civiweb
    except Exception as e:
        print(f"  ⚠️  Erreur pour l'offre {offer_id}: {e}")
        continue
    
    print(f"✅ {len(all_chunks)} chunks créés\n")

# ÉTAPE 3: Créer les embeddings
print("\n[3/5] Création des embeddings OpenAI...")
chunks_with_embeddings = create_embeddings_batch(all_chunks)


print("\n[4/5] Sauvegarde des données...")
save_to_json(chunks_with_embeddings, "vie_rag_data.json")
save_embeddings_numpy(chunks_with_embeddings, "vie_embeddings.npz")


# ÉTAPE 5: Statistiques
print("\n[5/5] Statistiques finales")
print("="*80)
print(f"Nombre d'offres traitées: {len(offer_ids[:10])}")
print(f"Nombre de chunks: {len(chunks_with_embeddings)}")
print(f"Dimension des embeddings: {len(chunks_with_embeddings[0]['embedding'])}")

# Calculer la taille totale
total_tokens = sum(len(c['content']) // 4 for c in chunks_with_embeddings)
print(f"Tokens estimés: ~{total_tokens:,}")
print(f"Coût estimé: ~${(total_tokens / 1_000_000) * 0.02:.4f}")
print("="*80)


PIPELINE RAG - OFFRES VIE

[1/5] Recherche des offres VIE...
Code de statut: 200
✅ 10 offres trouvées

[2/5] Récupération des détails et création des chunks...
  Traitement offre 1/10: 227934
   Récupération des données réussies
   Nettoyage des données
   Taille du chunk de Description (caractères) : 2907
   Taille du chunk de Profil (caractères) : 828
✅ 2 chunks créés

  Traitement offre 2/10: 230682
   Récupération des données réussies
   Nettoyage des données
   Taille du chunk de Description (caractères) : 1771
   Taille du chunk de Profil (caractères) : 615
✅ 4 chunks créés

  Traitement offre 3/10: 230189
   Récupération des données réussies
   Nettoyage des données
   Taille du chunk de Description (caractères) : 2624
   Taille du chunk de Profil (caractères) : 629
✅ 6 chunks créés

  Traitement offre 4/10: 230718
   Récupération des données réussies
   Nettoyage des données
   Taille du chunk de Description (caractères) : 1267
   Taille du chunk de Profil (caractères) : 1730
✅

# Now let's ask a question and embed it

In [12]:
question = "Quelle est la meilleur Offre pour faire de la Data"

In [13]:
question_embeddings = np.array([get_text_embedding(question)])

# Comparison bewtween our embedding vector and our embedding question

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
def find_closest_embedding(
    question_embeddings: np.array,
    text_embeddings: np.ndarray,
    top_n: int = 3,
) -> tuple[np.ndarray, np.ndarray]:

    # Calculer les similarités avec tous les embeddings
    similarites = cosine_similarity(question_embeddings, text_embeddings)[0]

    # Récupérer les top_n index et scores
    top_index = np.argsort(similarites)[-top_n:][::-1]  # Tri décroissant
    top_scores = similarites[top_index]

    return top_scores, top_index

In [18]:
loaded_embeddings = load_embeddings("vie_embeddings.npz")
text_embeddings = loaded_embeddings['embeddings']
top_scores, top_index = find_closest_embedding(question_embeddings, text_embeddings)

top_results = []

print("Top 3 embeddings les plus proches:\n ")
for score, idx in zip(top_scores, top_index):
    print(f"- Embedding {idx}: Score = {score:.4f}")
    print(f"metadata : {loaded_embeddings['metadata'][idx]}")
    print(f"contents : {loaded_embeddings['contents'][idx]}")
    top_results.append(loaded_embeddings['contents'][idx])

    # loaded_embeddings['metadata'][idx] PAS UTILISER POUR L'INSTANT

Top 3 embeddings les plus proches:
 
- Embedding 14: Score = 0.4515
metadata : {'offer_id': 'VIE231716', 'company': 'EQUANS', 'title': 'DATA ENGINEER HF', 'country': 'AUSTRALIE', 'city': 'SYDNEY                                  ', 'sector': 'ENERGIES', 'duration_months': 12, 'salary_eur': 3206, 'start_date': '2026-01-01T00:00:00', 'contact_email': 'Annabelle.eggerling@equans.com ', 'mission_type': 'VIE', 'chunk_type': 'mission_description', 'chunk_id': '230716_desc'}
contents : Offre VIE VIE231716 - DATA ENGINEER HF
    Entreprise: EQUANS
    Localisation: SYDNEY                                  , AUSTRALIE
    Secteur d'activité: ENERGIES
    Durée: 12 mois
    Indemnité: 3206 € par mois
    Description de la mission: Présentation de la société :
Equans is a world leader in the energy and services sector, with annual revenues of nearly €19,2 billion* and almost 800,000 projects. Equans has leading positions in Europe, which is the result of the history of energy construction in these 

# We Use a LLM to answer our question using the closest embedding

In [17]:

# Concaténer les top-N segments en un seul contexte
context = "\n".join(top_results)
prompt = f"Contexte :\n{context}\n\nQuestion : {question}\nRéponse :"


response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Tu es un assistant spécialisé dans la recherche d'information à partir de documents fournis. Tes réponses doivent absolument provenir du contexte fourni."},
        {"role": "user", "content": prompt}
    ]
)

print(response.choices[0].message.content)


La meilleure offre pour faire de la data est l'offre "VIE VIE231716 - DATA ENGINEER HF" chez EQUANS à Sydney, Australie. Cette offre est spécifiquement axée sur la mise en œuvre de l'écosystème data pour ANZ et le développement de logiciels embarquant des modèles variés, y compris la vision par ordinateur.
